# Fun with Trees

In this homework, let us experiment with creating and parsing expression trees in Julia.

In [23]:
tree(v::Tuple) = tree(v...)
tree(a,b) = throw("Need an odd number of arguments") # throw an error
tree(a) = throw("Need at least three arguments")
tree(a,b,c) = Expr(:call,a,b,c)
tree(v...) = Expr(:call,v[1],tree(v[2:end-1]),v[end])


tree (generic function with 5 methods)

In [24]:
@show tree(:*,:+,:^,:x,3,:y,4)
# Interchange args 2&3, and 5&6
@show tree(:*,:^,:+,:x,:y,3,4);
@show tree(:*,:^,:+,:x,3,:y,4);


tree(:*, :+, :^, :x, 3, :y, 4) = :((x ^ 3 + y) * 4)
tree(:*, :^, :+, :x, :y, 3, 4) = :((x + y) ^ 3 * 4)
tree(:*, :^, :+, :x, 3, :y, 4) = :((x + 3) ^ y * 4)


In [26]:
#using Pkg
#Pkg.add("Latexify")
using Latexify

See the [latexify and how Julia's metaprogramming makes it so useful](https://www.youtube.com/watch?v=wpV0Nz-93Hk) video from juliacon!

In [27]:
latexify(tree(:*,:+,:^,:x,3,:y,4) )

L"$\left( x^{3} + y \right) \cdot 4$"

In [28]:
latexify(tree(:*,:^,:+,:x,:y,3,4) )

L"$\left( x + y \right)^{3} \cdot 4$"

In [29]:
latexify(Expr(:call,:/,tree(:*,:+,:^,:x,3,:y,4),tree(:*,:^,:+,:x,:y,3,4)))

L"$\frac{\left( x^{3} + y \right) \cdot 4}{\left( x + y \right)^{3} \cdot 4}$"

## Exercise: 

Build a [reverse polish](https://en.wikipedia.org/wiki/Reverse_Polish_notation_) calculator in julia.

My high school math teacher [(Gil Kessler)](https://www.wnyc.org/story/302441-where-math-teachers-go-to-get-energized)
proudly showed me his reverse Polish HP calculator at a time when I had only heard of Texas Instruments.  I believe it was a model like this [HP 25](https://en.wikipedia.org/wiki/HP-25)
I thought reverse Polish was very strange, but he insisted it was way faster (fewer keystrokes).

In [214]:
# No error checking or input validation...

reverse_polish(a,b,c) = :($c($a, $b))
reverse_polish(v::Tuple) = reverse_polish(v...)
function reverse_polish(v...)
    op_idx = findfirst(isequal(Symbol), typeof.(v))
    op = v[op_idx]
    tmp_result = reverse_polish(v[op_idx-2:op_idx]...)
    v = v[1:op_idx-3]..., tmp_result, v[op_idx+1:end]...
    return reverse_polish(v)
end

reverse_polish (generic function with 4 methods)

In [215]:
using Test

@test reverse_polish(3,4,5,:*,:-) == :(3 - 4 * 5)
@test eval(reverse_polish(4,5,:*)) == 20
@test eval(reverse_polish(5,9,4,6,:+,:*,://)) == 1//18
@test convert(Int64, eval(reverse_polish(15,7,1,1,:+,:-,:/,3,:*,2,1,1,:+,:+,:-))) == 5

Test Passed

In [219]:
@show reverse_polish(3,4,5,:*,:-);
@show eval(reverse_polish(3,4,5,:*,:-));
@show reverse_polish(4,5,:*);
@show eval(reverse_polish(4,5,:*));
@show reverse_polish(5,9,4,6,:+,:*,://);
@show eval(reverse_polish(5,9,4,6,:+,:*,://));
@show reverse_polish(15,7,1,1,:+,:-,:/,3,:*,2,1,1,:+,:+,:-);
@show eval(reverse_polish(15,7,1,1,:+,:-,:/,3,:*,2,1,1,:+,:+,:-));

reverse_polish(3, 4, 5, :*, :-) = :(3 - 4 * 5)
eval(reverse_polish(3, 4, 5, :*, :-)) = -17
reverse_polish(4, 5, :*) = :(4 * 5)
eval(reverse_polish(4, 5, :*)) = 20
reverse_polish(5, 9, 4, 6, :+, :*, ://) = :(5 // (9 * (4 + 6)))
eval(reverse_polish(5, 9, 4, 6, :+, :*, ://)) = 1//18
reverse_polish(15, 7, 1, 1, :+, :-, :/, 3, :*, 2, 1, 1, :+, :+, :-) = :((15 / (7 - (1 + 1))) * 3 - (2 + (1 + 1)))
eval(reverse_polish(15, 7, 1, 1, :+, :-, :/, 3, :*, 2, 1, 1, :+, :+, :-)) = 5.0
